# Introduction to Dynamic Programming

## CSCI E-82A
## Stephen Elston

In the previous lesson we explored the concepts of **Markov decision processes (MDP)** and **Markov reward processes (MRP)**. Now we will extend these concepts and apply them to finding **optimal solutions** for such systems. By an optimal solution, we mean a solution which produces **greater reward** than any other solution. 

To understand how we can find optimal solutions we must introduce some new concepts:
1. An **action** causes a state transition. This transition may be to the same state. 
2. A **policy** specifies the **actions** given the state. In other words, the policy defines the actions to be specified by the agent. 
3. An **optimal policy** produces the greatest reward possible given the initial state of the system.
4. A **plan** is the sequence of actions leading to an optimal result. 

In this an subsequent lessons, we will explore some powerful methods for finding optimal policies. Broadly, these methods are known as **dynamic programming** and **reinforcement learning**. In this lesson we will focus on the representation and learning methods for dynamic programming. Dynamic programming algorithms can be the basis of effective and flexible intelligent agents as shown below.

<img src="img/DPAgent.JPG" alt="Drawing" style="width:500px; height:300px"/>
<center> **Dynamic Programming Agent and Environment** </center>  


**Suggested readings** for dynamic programming are:
- Sutton and Barto, second edition, Sections 3.7, 3.8, and Chapter 4 or,   
- Russell and Norvig, third edition, Sections 17.1, 17.2 or, 
- Kochenderfer, Sections 4.1, 4.2, 4.3, 4.5, 4.6.

## What is Dynamic Programming?

What is **dynamic programming**? In the most general terms, dynamic programming is a **optimal planning method**. Planning methods enable an intelligent agent to gain improved autonomy though a sequence of optimal actions to achieve a **goal**. 

Dynamic programming was developed in the 1950's by mathematician [**Richard Bellman**](https://en.wikipedia.org/wiki/Richard_E._Bellman). By *programming* Bellman meant a computer algorithm which computes a a plan of actions to optimize the **utility** or **total reward** for the states visited in a Markov process. By *dynamic*, Bellman meant the algorithm solves the problem recursively by operating on smaller and simpler sub-problems. 

Like dynamic programming, **reinforcement learning** is class of optimization algorithms using a sequence of actions for a system represented by a Markov processes.Therefore, understanding dynamic programming is good path to understanding reinforcement learning. 

. **Note:** If you are familiar with machine learning, you may recognize Bellman as the person who coined the phrase *the curse of dimensionality*.

## Value Functions and Policy Evaluation

Given a Markov random process, a reward function, and a policy of actions, how can we evaluate the policy? We can perform **policy evaluation** in two ways, using a value function or an action value function.  

### Policy Evaluation with Value Functions

First, we can perform **policy evaluation** with a **value function**. The value function is the expected value of the **gain** achieved by following a policy, $\pi(a|s)$, given the current state. We can express the state value function as follows:

$$
\begin{align}
v_{\pi}(s) &= \mathbb{E}_{\pi} [ G_t\ |\ S_t = s] \\
&= \mathbb{E}_{\pi} [R_{t+1} + \gamma G_{t+1}\ |\ S_t = s] \\
&= \mathbb{E}_{\pi} [R_{t+1} + \gamma v_{\pi} (S_{t+1})\ |\ S_t = s] \\
&= \sum_a \pi(a|s) \sum_{s',r} p(s',r | s,a) \big[ r + \gamma v_{\pi}(s') \big],\ \forall a 
\end{align}
$$
Where,  
$\mathbb{E}_{\pi} [] =$ Expectation given policy $\pi$,   
$v_{\pi}(s) =$ value of state, s, given policy $\pi$,   
$G_t =$ return at step $t$,    
$S_t =$ state at step $t$,    
$R_t =$ reward at step $t$,  
$\pi(a|s) =$ the policy specifying action, $a$, given state, $s$,  
$p(s',r | s,a) =$ probability of successor state, $s'$, and reward, $r$, given state, $s$, and action $a$,  
$\gamma =$ discount rate. 


The above relations are known as the **Bellman value equations**. This relationship tells us how to compute the value of being in a particular state, $s$. There is one such equation for each state, $s \in \xi$, of the Markov process.

In principle, we could find the state values by solving the $\xi \in R^n$ simultaneous linear Bellman state value equations. However, this approach has computational complexity $O(n^3)$. 

Examine the last line of the above relations and notice it can be viewed as a **recursion**. This leads us to an iterative solutions. We can start with an initial set of state values (e.g, 0) and iterate the equation through $T$ time steps, until meeting a **convergence criteria**. The convergence criteria is a measure in the change of the state values from one iteration to the next. 

The recursion of the state value equations iterate though values of the successor states. The estimate of state value is improved at each step using= the estimates of successor states. We call such an algorithm a **Bootstrap method**. 



### Policy Evaluation with Action Value Functions

As an alternative to the state value function, we can use the **Bellman state action function**:

$$\begin{align}
q_\pi(s,a) &= \mathbb{E_\pi} \big[G_{t}\ \big|\ S_t = s, A_t = a \big]  \\
&= \mathbb{E_\pi} \Big[ \sum_{k=0}^\infty \gamma^k\ R_{t+k+1} \big|\ S_t = s, A_t = a \Big]  \\
&= \sum_{s',r} p(s',r\ |\ s,a) \big[ r + \gamma\ q_\pi(S_{t+1},a') \big]
\end{align}$$

Where,
$q_\pi(s,a) =$ is the action value, which is the value of action, $a$ from state $s$ following policy $\pi$,   
$A_t = $ the action taken at step $t$,  
$a' = $ the action taken from the successor state, $s'$,  
and other notation is the same as discussed for the state value function. 


Examining the last line of the above relationship shows that this is a recursion relation as well. Thus, we can bootstrap with the action value function to iteratively find the action values for the Markov process. 

## State Value for Grid World Example

Let's try an example of computing the state values of a Markov process. In this example, we will work with both the **representations** and **learning**. We will apply the state value function approach here. 

**Navigation** to a goal is a significant problem in robotics. Real-world navigation is rather complex. Therefore, in this example we will use a simple analog called a **grid world**. The grid world for this problem is shown below. 

<img src="img/GridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **A 4x4 Grid World with Terminal State** </center>

The grid world consists of a 4x4 set of positions the robot can occupy. Each position is considered a state. The goal is to navigate to state 0, the goal, in the minimum steps. We will explore methods to find policies which reach this goal and achieve maximum reward. 

Grid position 0 is the goal and a **terminal state**. There are no possible state transitions out of this position. The presence of a terminal state makes this an **episodic Markov random process**. In each episode the robot can start in any other random position, $\{ 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 \}$, and the episode terminates when the robot enters the terminal position (state 0).  

In each state, there are four possible actions the robot can take:
- up, u
- down, d,
- left, l
- right, r

We encode, or represent, these possibilities in a dictionary as shown in the code block below. We use a dictionary of dictionaries to perform the lookup. The keys of the outer dictionary are the identifiers (numbers) of the states. The keys of the inner dictionary are the possible actions and the values are the **successor state**, $s'$, for that transition.  

Notice that there are no allowed transitions out of the terminal state. Also, any transition that might take the robot off the grid, leaves the state unchanged. 

In [2]:
## import numpy for latter
import numpy as np

## Define the transition dictonary of dictionaries:
policy = {0:{'u':0, 'd':0, 'l':0, 'r':0},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':9, 'd':13, 'l':12, 'r':14},
          14:{'u':10, 'd':14, 'l':13, 'r':15},
          15:{'u':11, 'd':15, 'l':14, 'r':15}}

We need to define the initial transition probabilities for the Markov process. We set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. As there are 4 possible transitions from each state, this means all transition probabilities are 0.25. In other words, this is a random policy which does not favor any particular plan. 

The initial uniform transition probabilities are encoded using a dictionary of dictionaries. The organization of this data structure is identical to the foregoing data structure. 

In [1]:
state_to_state_probs = {0:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}}


The robot receives the following rewards:
- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

We encode this reward in the same type of dictionary structure used for the foregoing structures.  

In [3]:
rewards = {0:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0},
          1:{'u':-1, 'd':-0.1, 'l':10.0, 'r':-0.1},
          2:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          4:{'u':10.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          5:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          8:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          10:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          11:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
          12:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
          13:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          14:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-0.1},
          15:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}}

So far, we have constructed a rather poor policy, which is just a random walk around the grid. Still, we can still measure the value of this policy. 

The function in the code below iterates over the Bellman value function to find the values of each state. The iteration continues until the convergence criteria is meet.  

> **Note:** The code in this example takes advantage of the fact that there is only one possible successor state for each action. This means there is no need to sum over successor states.

In [4]:
def compute_state_value(pi, probs, reward, gamma = 1.0, theta = 0.01, display = False):
    '''Function for policy evaluation  
    '''
    delta = theta
    values = np.zeros(len(probs)) # Initialize the value array
    while(delta >= theta):
        v = np.copy(values) ## save the values for computing the difference later
        for s in probs.keys():
            temp_values = 0.0 ## Initial the sum of values for this state
            for action in rewards[s].keys():
                s_prime = pi[s][action]
                temp_values = temp_values + probs[s][action] * (reward[s][action] + gamma * values[s_prime])
            values[s] = temp_values
            
        ## Compute the difference metric to see if convergence has been reached.    
        diffs = np.sum(np.abs(np.subtract(v, values)))
        delta = min([delta, diffs])
        if(display): 
            print('difference metric = ' + str(diffs))
            print(values.reshape(4,4))
    return values

compute_state_value(policy, state_to_state_probs, rewards, theta = 0.1, display = True).reshape(4,4)

difference metric = 9.65546875
[[ 0.          2.2         0.225      -0.49375   ]
 [ 2.2         1.          0.20625    -0.396875  ]
 [ 0.225       0.20625     0.003125   -0.4234375 ]
 [-0.49375    -0.396875   -0.4234375  -0.76171875]]
difference metric = 5.476733398437499
[[ 0.          3.05625     0.4234375  -0.79023437]
 [ 3.05625     1.53125     0.29023437 -0.65507812]
 [ 0.4234375   0.29023437 -0.16660156 -0.82670898]
 [-0.79023438 -0.65507813 -0.82670898 -1.34421387]]
difference metric = 3.4751190185546883
[[ 0.          3.45273438  0.51904297 -0.97912598]
 [ 3.45273438  1.77148438  0.26721191 -0.87342529]
 [ 0.51904297  0.26721191 -0.37974854 -1.18102417]
 [-0.97912598 -0.87342529 -1.18102417 -1.81261902]]
difference metric = 2.5667635917663567
[[ 0.          3.63581543  0.53573608 -1.12398529]
 [ 3.63581543  1.85151367  0.18351898 -1.07372894]
 [ 0.53573608  0.18351898 -0.59875259 -1.49153118]
 [-1.12398529 -1.07372894 -1.49153118 -2.2020751 ]]
difference metric = 2.18674486875

array([[ 0.        ,  0.92461824, -3.93184927, -6.48887002],
       [ 0.92461824, -2.10995108, -4.95119803, -6.86735717],
       [-3.93184927, -4.95119803, -6.5102566 , -7.87700873],
       [-6.48887002, -6.86735717, -7.87700873, -8.96905736]])

The array displayed above shows the value of being in a state (grid position) given the policy. You can see that the closer the position is to the goal, the higher the value. This intuitively makes sense.  

## Overview of Dynamic Programming - Policy Iteration

We now have a method to evaluate a policy for a Markov process. But, the current random policy is rather poor. Fortunately, the Bellman equations give us several possible dynamic programming algorithms for **policy improvement**. 

Before we get too far here, we should define what exactly mean by policy improvement. Intuitively, an improved policy should yield greater total utility than a previous policy. More specifically an improved policy $\pi'$ has the following relationship with an initial policy, $\pi$:

$$v_{\pi'}(s) >= v_\pi(s)$$

We can also say that an optimal policy $\pi_*$ must conform to the following:

$$v_{*}(s) >= v_\pi(s)\ \forall \pi$$

In words, the optimal policy has a value greater than or equal to all other possible policies. Notice there is **no guarantee of uniqueness**. There can be multiple optimal policies.   

The key idea of dynamic programming is expressed by the **Bellman optimality equations**. There are two ways we can express this optimal relationship. We can find a relationship which is a solution to the **Bellman optimal state action equations**, denoted $q_{*}(s,a)$. Or, we can find a solution to the **Bellman optimal state value equations**, $v_{*}(s)$, shown here:

$$
\begin{align}
v_{*}(s) &= max_\pi v_\pi(s) \\
&= max_a\ \mathbb{E} [ G_{t+1} + \gamma v_*(S_{t+1})\ |\ S_t = s, A_t = a] \\
&= max_a\ \sum_{s',r} p(s',r\ |\ s,a) \big[ r + \gamma v_{*}(s') \big]
\end{align}
$$

Given the $max_a$ operation this equations are no longer linear. But as with the state value functions, the optimal state value equations are naturally recursive and amenable to iterative solutions.

We can gain some insight into the iterative process for the Bellman optimal state value equations by studying their **backup diagram**. A backup diagram is read from top to bottom. States are shown as open circles and actions by filled circles. The backup diagram for the Bellman optimal state value iteration is shown below.

<img src="img/ValueBackup.JPG" alt="Drawing" style="width:250px; height:200px"/>
<center> **Backup diagram of Bellman Optimal State Value Function** </center>

Let's follow this backup diagram from top to bottom.
- The Markov process starts in some initial state $s$. 
- We take the action $a$ that maximizes the state value.
- This leads to successor states, $s'$ with reward $r$.

Notice that this algorithm, like a dynamic programming methods, requires us to evaluate the value of all possible actions for a given state. You can see this is the way the max sweep is shown in the above diagram. We say that dynamic programming requires **full backups**. This fact puts a lower bound on the computational complexity of dynamic programming methods. 

The requirement to perform full backups lead Richard Bellman to declare the *curse of dimensionality*. That is as the number of states (dimensions) grows so goes the computational complexity of dynamic programming. None the less, dynamic programming is a surprisingly scalable method. We have seen how DP is much more efficient that direct solution methods. Further, DP algorithms have significantly lower complexity than the equivalent graph search algorithms or linear programming methods. 

## Policy Iteration for Grid World

Let's try the policy iteration algorithm on the grid world example. The function in the cell below performs policy iteration by the following steps:
1. The state values and policy are initialized to some arbitrary starting point.
2. The policy with the maximum value is found for all possible actions by bootstrapping.
3. The improvement in the policy is evaluated using the state value function to determine if convergence has been achieved. If not, continue with step 2.

The code in the cell below implements this algorithm and applies it to the grid world. Execute this code and examine the result. 

In [5]:
import copy
def policy_iteration(pi, probs, reward, gamma = 1.0, theta = 0.1, output = False):
    delta = theta
    v = np.zeros(len(probs))
    state_values = np.zeros(len(probs))
    current_policy = copy.deepcopy(probs)
    while(delta >= theta): # Continue until convergence.
        for s in probs.keys(): # Iterate over all states
            temp_values = []
            for action in rewards[s].keys(): # Iterate over all possible actions for the state
                # Compute list of values given action from the state
                s_prime = pi[s][action]
                temp_values.append(current_policy[s][action] * (reward[s][action] + gamma * state_values[s_prime]))
            
            ## Find the max value and update current policy
            max_index = np.where(np.array(temp_values) == max(temp_values))[0]
            prob_for_policy = 1.0/float(len(max_index))
            for i,action in enumerate(current_policy[s].keys()): 
                if(i in max_index): current_policy[s][action] = prob_for_policy
                else: current_policy[s][action] = 0.0
                
        
        ## Compute state values with new policy to determine if there is an improvement
        ## Uses the compute_state_value function
        state_values = compute_state_value(pi, current_policy, rewards, theta = .1)
        diff = np.sum(np.abs(np.subtract(v, state_values)))
        if(output): 
            print('\ndiff = ' + str(diff))
            print('Current policy')
            print(current_policy)
            print('With state values')
            print(state_values.reshape(4,4))
        
        delta = min([delta, np.sum(np.abs(np.subtract(v, state_values)))])
        v = np.copy(state_values) ## copy of state values to evaluate next iteration
    return current_policy

policy_iteration(policy, state_to_state_probs, rewards, gamma = 1.0, output = True)


diff = 128.1086520299247
Current policy
{0: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}, 1: {'u': 0.0, 'd': 0.0, 'l': 1.0, 'r': 0.0}, 2: {'u': 0.0, 'd': 0.3333333333333333, 'l': 0.3333333333333333, 'r': 0.3333333333333333}, 3: {'u': 0.0, 'd': 0.5, 'l': 0.5, 'r': 0.0}, 4: {'u': 1.0, 'd': 0.0, 'l': 0.0, 'r': 0.0}, 5: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}, 6: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}, 7: {'u': 0.3333333333333333, 'd': 0.3333333333333333, 'l': 0.3333333333333333, 'r': 0.0}, 8: {'u': 0.3333333333333333, 'd': 0.3333333333333333, 'l': 0.0, 'r': 0.3333333333333333}, 9: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}, 10: {'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}, 11: {'u': 0.3333333333333333, 'd': 0.3333333333333333, 'l': 0.3333333333333333, 'r': 0.0}, 12: {'u': 0.5, 'd': 0.0, 'l': 0.0, 'r': 0.5}, 13: {'u': 0.3333333333333333, 'd': 0.0, 'l': 0.3333333333333333, 'r': 0.3333333333333333}, 14: {'u': 0.3333333333333333, 'd': 0.0, 'l': 0.3333333333333333, 'r': 0.3333333

{0: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 1: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 2: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 3: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 4: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 5: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 6: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 7: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 8: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 9: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 10: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 11: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 12: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 13: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 14: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 15: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5}}

How can we interpret this optimal plan? The plan specifies the optimal state transitions to travel from any state to the goal. For example, the diagram below shows the optimal paths from State 15 to the goal. Notice that these paths are not unique in this plan. 

<img src="img/OptimalPath.JPG" alt="Drawing" style="width:300px; height:300px"/>
<center> **Optimal Paths From State 15 to Goal** </center>

Any of the 4 possible alternatives illustrated above are optimal:

- 15 -> 11 -> 7 -> 6 -> 5 -> 1 -> 0
- 15 -> 11 -> 7 -> 6 -> 5 -> 4 -> 0
- 15 -> 14 -> 13 -> 9 -> 5 -> 1 -> 0
- 15 -> 14 -> 13 -> 9 -> 5 -> 4 -> 0

## Value Iteration for Policy Improvement

Now, we will investigate using the Bellman **optimal state action equations** for policy improvement. In this context, the policy improvement relations are expressed as:

$$q_{\pi}(s, \pi_{*}(s)) >= v_\pi(s) \forall \pi$$

In words, the optimal state action policy $\pi_{*}(s)$ gives a value greater than or equal to all other state action policies. Once again, there is no guarantee of uniqueness. 

The Bellman optimal state action equations are expressed:

$$\begin{align}
q_{*}(s,a) &= max_\pi q(s,a)\\
&= \mathbb{E} \big[R_{t+1} + \gamma max_{a'}\ q_{*}(S_{t+1},a')\ 
\big|\ S_t = s, A_t = a \big]  \\
&= max_a \sum_{s',r} p(s',r\ |\ s,a) \big[ r + \gamma\ max_{a'}\ q_{*}(S_{t+1},a') \big]
\end{align}$$

Where, $q_{*}(s,a)$ is the policy giving action, $a$, from a state, $s$ with an **optimal policy**, $\pi_*$. As with the Bellman state action value equations, there is one such equation for each action value tuple, $(a,s)$.


We can gain some understanding of how the recursion works by looking at the **backup diagram** shown below.

<img src="img/ActionValueBackup.JPG" alt="Drawing" style="width:250px; height:200px"/>
<center> **Backups diagram of Bellman Optimal Action Value Function** </center>


Starting at the top of the diagram:
- The Markov process starts with a state-action tuple, $(s,a)$.
- This state-action leads to successor states, $s'$ with reward $r$.
- The successor action, $a'$, with the maximum expect value is selected. 

As with policy iteration, notice that value iteration requires a full backup at each step. 

## Value Iteration for Grid World

Let's try the value iteration algorithm on the grid world example. The function in the cell below performs value iteration by the following steps:
1. The state values and policy are initialized to some arbitrary starting point.
2. For possible state actions, the maximum value for successor states is found.
3. Policy improvement is measures to find if the algorithm has converged. If not steps 2 and 3 are repeated. 
3. Given the resulting optimal state actions, the optimal policy $\pi_*$ is found by taking the maximum over possible actions for each successor state.

The code in the cell below implements this algorithm and applies it to the grid world. Execute this code and examine the result. 

In [6]:
def value_iteration(pi, probs, reward, gamma = 1.0, theta = 0.1, output = False):
    delta = theta
    v = np.zeros(len(probs))
    state_values = np.zeros(len(probs))
    current_policy = copy.deepcopy(probs)
    while(delta >= theta):
        for s in probs.keys(): # iteratve over all states
            temp_values = []
            ## Find the values for all possible actions in the state.
            for action in rewards[s].keys():
                s_prime = pi[s][action]
                temp_values.append((reward[s][action] + gamma * state_values[s_prime]))
            
            ## Find the max value and update the value for the state
            state_values[s] = max(temp_values)
        ## Determine if convergence is achieved
        diff = np.sum(np.abs(np.subtract(v, state_values)))
        delta = min([delta, np.sum(np.abs(np.subtract(v, state_values)))])
        v = np.copy(state_values)
        if(output):
            print('Difference = ' + str(diff))
            print(state_values.reshape(4,4))
    
    ## Now we need to find the policy that makes max value state transitions
    for s in current_policy.keys(): # iterate over all states
        ## Find the indicies of maximum state transition values
        temp_values = [state_values[pi[s][s_prime]] for s_prime in pi[s].keys()]
        max_index = np.where(np.array(temp_values) == max(temp_values))[0]    
        prob_for_policy = 1.0/float(len(max_index)) ## Probabilities of transition
        for i, key in enumerate(current_policy[s]): ## Update policy
            if(i in max_index): current_policy[s][key] = prob_for_policy
            else: current_policy[s][key] = 0.0    
    return current_policy

value_iteration(policy, state_to_state_probs, rewards, output = True)

Difference = 146.70000000000002
[[ 0.  10.   9.9  9.8]
 [10.   9.9  9.8  9.7]
 [ 9.9  9.8  9.7  9.6]
 [ 9.8  9.7  9.6  9.5]]
Difference = 0.0
[[ 0.  10.   9.9  9.8]
 [10.   9.9  9.8  9.7]
 [ 9.9  9.8  9.7  9.6]
 [ 9.8  9.7  9.6  9.5]]


{0: {'d': 0.25, 'l': 0.25, 'r': 0.25, 'u': 0.25},
 1: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 2: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 3: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 4: {'d': 0.0, 'l': 1.0, 'r': 0.0, 'u': 0.0},
 5: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 6: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 7: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 8: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 9: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 10: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 11: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 12: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 1.0},
 13: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 14: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5},
 15: {'d': 0.0, 'l': 0.5, 'r': 0.0, 'u': 0.5}}

The resulting optimal plan is not the same as the one developed by policy iteration. There are more options, However, the utility of an set of state transitions from the initial state the goal will be the same.  

## Generalized Policy Improvement

In the foregoing we have been using full backups for the dynamic programming algorithms. There is an alternative which can be both more efficient and can result in faster convergence. This concept is known as **generalized policy improvement** or GPI. GPI is applicable to both dynamic programming and reinforcement learning.

The general idea of GPI is to break the policy improvement and evaluation steps into opposing processes and iterate between them. This process can be done in a quite granular way, even on a single state at a time. This makes the policy improvement algorithms much more scalable. The figure below illustrates the concept of GPI.  

<img src="img/GPI.JPG" alt="Drawing" style="width:250px; height:250px"/>
<center> **Concept of Generalized Policy Improvement** </center>

In GPI the policy improvement step makes the evaluation step less accurate and vice versa. This is what we mean by these steps acting in opposition. Over a number of iterations the gap between policy improvement and evaluation becomes narrower and eventually converges.  

#### Copyright 2018, Stephen F Elston. All rights reserved. 